In [14]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation
import matplotlib
from matplotlib.patches import Polygon
from decimal import Decimal
import copy


# matplotlib.use('Agg')
matplotlib.use('Qt5Agg')
# First set up the figure, the axis, and the plot element we want to animate
# fig = plt.figure()
# ax1 = plt.subplot(1, 1, 1)
# ax2 = plt.subplot(1, 2, 1)
fig, axes = plt.subplots(1, 5, frameon=False)
axes[0].set_position([0.05, 0.05, 0.3, 0.4])
axes[1].set_position([0.4, 0.05, 0.2, 0.9])
axes[2].set_position([0.05, 0.55, 0.3, 0.4])
axes[3].set_position([0.7, 0.05, 0.3, 0.4])
axes[4].set_position([0.7, 0.55, 0.3, 0.4])

# ax = plt.axes(xlim=(0, 2), ylim=(-2, 2))
# line, = ax.plot([], [], lw=2)

pst_w = 5
res_down_c = 7
res_hight = 18
pst_thick = 2
pst_rog = 1
res_thick = 0.5
nozle_d = 0.8
nozle_h = 1
res_down_offset = 1
res_up_offset = 1
res_up_hole = 1.5
res_up_h = 6

vent_w = 2
vent_t = 0.5
vent_r = 0.6
vent_h = 3


nozzle_coords = []
n_grad_steps = 10
nozzle_y_coords = np.linspace(-nozle_h-res_thick, 0, n_grad_steps)

for i in range(nozzle_y_coords.shape[0]-1):
    nozzle_coords.append(
        np.array([
            [-nozle_d/2, nozzle_y_coords[i]],
            [-nozle_d/2, nozzle_y_coords[i+1]],
            [nozle_d/2, nozzle_y_coords[i+1]],
            [nozle_d/2, nozzle_y_coords[i]]
            ])
    )

vent_coords = np.array([
    [-vent_w/2, res_hight-vent_t],
    [-vent_w/2, res_hight],
    [-vent_r/2, res_hight],
    [-vent_r/2, res_hight+vent_h],
    [vent_r/2, res_hight+vent_h],
    [vent_r/2, res_hight],
    [vent_w/2, res_hight],
    [vent_w/2, res_hight-vent_t]
])

reservars_coords = [
    np.array([
        [-nozle_d/2, 0],
        [-nozle_d/2, -nozle_h-res_thick],
        [-nozle_d/2-res_thick, -nozle_h-res_thick],
        [-nozle_d/2-res_thick, -res_thick],
        [-pst_w/2-res_thick, -res_thick],
        [-pst_w/2-res_thick, res_down_c+res_down_offset],
        [-pst_w/2, res_down_c+res_down_offset],
        [-pst_w/2, 0],
    ]),
    np.array([
        [nozle_d/2, 0],
        [nozle_d/2, -nozle_h-res_thick],
        [nozle_d/2+res_thick, -nozle_h-res_thick],
        [nozle_d/2+res_thick, -res_thick],
        [pst_w/2+res_thick, -res_thick],
        [pst_w/2+res_thick, res_down_c+res_down_offset],
        [pst_w/2, res_down_c+res_down_offset],
        [pst_w/2, 0],
    ]),
    np.array([
        [-res_up_hole/2, res_hight],
        [-pst_w/2, res_hight],
        [-pst_w/2, res_hight-res_up_h],
        [-pst_w/2-res_thick, res_hight-res_up_h],
        [-pst_w/2-res_thick, res_hight+res_thick],
        [-res_up_hole/2, res_hight+res_thick],
    ]),
    np.array([
        [res_up_hole/2, res_hight],
        [pst_w/2, res_hight],
        [pst_w/2, res_hight-res_up_h],
        [pst_w/2+res_thick, res_hight-res_up_h],
        [pst_w/2+res_thick, res_hight+res_thick],
        [res_up_hole/2, res_hight+res_thick],
    ])
]

n_step = 100

# time = np.linspace(0, 10, n_step)
time = np.arange(n_step)
x_p_vec = np.linspace(1, 5, n_step)
p_up_vec = np.linspace(9, 1, n_step)**2
p_down_vec = np.linspace(1, 9, n_step)**2
x_th_vec = np.linspace(0, 1.5, n_step)
l_jet_vec = np.linspace(0, 5, n_step)
v_jet_vec = np.ones(n_step)/5
l_critic = 1.5

p_down_min = p_down_vec.min()
p_down_max = p_down_vec.max()


min_p_down_id = np.argmin(p_down_vec)

# cmap = plt.cm.rainbow

# up_norm = matplotlib.colors.Normalize(vmin=p_up_vec.min(), vmax=p_up_vec.max)
# down_norm = matplotlib.colors.Normalize(vmin=p_down_vec.min(), vmax=p_down_vec.max)

cmap = matplotlib.colormaps['plasma']
down_colors = cmap(p_down_vec/p_down_vec.max()) 
up_colors = cmap(p_up_vec/p_up_vec.max()) 
min_p_down_color = down_colors[min_p_down_id]

plot_dict = {0: {'data': l_jet_vec, 'color': 'red', 'y_tit': 'l_jet, mm'},
             2: {'data': p_down_vec, 'data2': p_up_vec, 'color': 'red', 'color2': 'red', 'y_tit': 'P_down/P_up, Pa'},
             3: {'data': x_th_vec, 'color': 'green', 'y_tit': 'x_th_vec, mm'},
             4: {'data': v_jet_vec, 'color': 'blue', 'y_tit': 'v_jet, m/c'}
             }


for plot_name in plot_dict:
    plot_dict[plot_name]['x_max'] = plot_dict[plot_name]['data'].shape[0]
    if 'data2' in plot_dict[plot_name]:
        y_min = min(plot_dict[plot_name]['data'].min(), plot_dict[plot_name]['data2'].min())
        y_max = max(plot_dict[plot_name]['data'].max(), plot_dict[plot_name]['data2'].max())

    else:
        y_min = plot_dict[plot_name]['data'].min()
        y_max = plot_dict[plot_name]['data'].max()

    hight = y_max - y_min
    y_min -= 0.05*hight
    y_max += 0.05*hight

    plot_dict[plot_name]['y_min'] = y_min
    plot_dict[plot_name]['y_max'] = y_max



drops_time = []
drops_init_vel = []
# circles_drops = []

def get_circles(rad=0.3):

    global drops_time
    global drops_init_vel
    # global circles_drops

    coords = [v*t + 1*(t**2)/2 for v, t in zip(drops_init_vel, drops_time)]

    y0 = -nozle_h-res_thick-l_critic
    true_coords = [y0 - y for y in coords]
    return [plt.Circle((0, y), rad, color=min_p_down_color) for y in true_coords]

def get_polygon(x_p, res_type, l_jet=None, x_th=None, p_down=None, c_up=None, c_down=None):
    if res_type == 'down':
        polygon_coords = np.array([
            [-pst_w/2,0],
            [-pst_w/2, res_down_c-x_p],
            [pst_w/2, res_down_c-x_p],
            [pst_w/2,0],
            ])
        return Polygon(polygon_coords, facecolor = c_down)
    
    elif res_type == 'up':
        polygon_coords = np.array([
            [-pst_w/2, res_hight-x_p],
            [-pst_w/2, res_hight],
            [pst_w/2, res_hight],
            [pst_w/2, res_hight-x_p]
            ])
        
        return Polygon(polygon_coords, facecolor = c_up)
    
    elif res_type == 'nozle':
        nozle_p_vec = np.linspace(p_down_min, p_down, n_grad_steps)
        # nozle_p_vec.append(p_down_max)

        nozle_colors = cmap(nozle_p_vec/p_down_max)

        polygons = []
        for j, polygon_coords in enumerate(nozzle_coords):
            polygons.append(Polygon(polygon_coords, facecolor = nozle_colors[j]))

        return polygons
    
    elif res_type == 'jet':
        polygon_coords = np.array([
            [-nozle_d/2, -nozle_h-res_thick-l_jet],
            [-nozle_d/2, -nozle_h-res_thick],
            [nozle_d/2, -nozle_h-res_thick],
            [nozle_d/2, -nozle_h-res_thick-l_jet]
            ])
        
        return Polygon(polygon_coords, facecolor = min_p_down_color)
    
    elif res_type == 'vent':
        
        polygon_coords = np.copy(vent_coords)
        polygon_coords[:, 1] = polygon_coords[:, 1] - x_th

        return Polygon(polygon_coords, facecolor = 'black')
 
    elif res_type == 'piston':
        polygon_coords = np.array([
            [-pst_w/2, res_down_c-x_p],
            [-pst_w/2, res_down_c-x_p+pst_thick],
            [-pst_rog/2, res_down_c-x_p+pst_thick],

            [-pst_rog/2, res_hight-x_p-pst_thick],
            [-pst_w/2, res_hight-x_p-pst_thick],
            [-pst_w/2, res_hight-x_p],

            [pst_w/2, res_hight-x_p],
            [pst_w/2, res_hight-x_p-pst_thick],
            [pst_rog/2, res_hight-x_p-pst_thick],

            [pst_rog/2, res_down_c-x_p+pst_thick],
            [pst_w/2, res_down_c-x_p+pst_thick],
            [pst_w/2, res_down_c-x_p]
            ])
        
        return Polygon(polygon_coords, facecolor = 'k')
    
    elif res_type == 'reservars':
        polygons = [Polygon(polygon_coords, facecolor = 'blue') for polygon_coords in reservars_coords]
        return polygons
    
    elif res_type == 'background':
        polygon_coords = np.array([
          [-pst_w/2-10, -20],
          [-pst_w/2-10, res_hight + 2],
          [pst_w/2+10, res_hight + 2],
          [pst_w/2+10, -20],
        ])
        return Polygon(polygon_coords, facecolor = 'white')

# initialization function: plot the background of each frame
def init():
    for j, ax in enumerate(axes):
        ax.clear()

    axes[1].axis('off')
    axes[1].set(frame_on=False)
    axes[1].tick_params(left = False, right = False , labelleft = False , 
            labelbottom = False, bottom = False)  
    #     # ax2.clear()
    #     ax.set_xlim(times[0], times[-1])
    #     # ax2.set_xlim(times[0], times[-1])

    #     ax.set_ylim(features[:, j].min(), features[:, j].max())
    #     # ax2.set_ylim(states[:, 1].min(), states[:, 1].max())
    #     ax.set(ylabel=states_name[j])
    
    axes[1].clear()
    axes[1].set_xlim([-pst_w/2 - 2, pst_w/2 + 2])
    axes[1].set_ylim([ - 15, res_hight + 2])
    
    global drops_time
    global drops_init_vel

    drops_time = []
    drops_init_vel = []

    return axes

# ort matplotlib.pyplot as plt
import matplotlib.collections as mcoll
def make_segments(x, y):
    """
    Create list of line segments from x and y coordinates, in the correct format
    for LineCollection: an array of the form numlines x (points per line) x 2 (x
    and y) array
    """

    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    return segments

# animation function.  This is called sequentially
def animate(i):
    for j, ax in enumerate(axes):
        
        ax.clear()

        if j != 1:
            axes[j].set_xlim([-0.5, plot_dict[j]['x_max']])
            axes[j].set_ylim([plot_dict[j]['y_min'], plot_dict[j]['y_max']])
            axes[j].set(ylabel=plot_dict[j]['y_tit'])

            if 'data2' in plot_dict[j]:
                axes[j].plot(time[:i], plot_dict[j]['data'][:i], c=plot_dict[j]['color'])
                # segments = make_segments(time[:i], plot_dict[j]['data'][:i])
                # lc = mcoll.LineCollection(segments, cmap=plot_dict[j]['color'][:i])
                # ax = plt.gca()
                # ax.add_collection(lc)
                
                axes[j].plot(time[:i], plot_dict[j]['data2'][:i], c=plot_dict[j]['color2'])

            else:
                axes[j].plot(time[:i], plot_dict[j]['data'][:i], c=plot_dict[j]['color'])

    global drops_time
    global drops_init_vel

    
    axes[1].axis('equal')
    axes[1].set_xlim([-pst_w/2 - 2, pst_w/2 + 2])
    axes[1].set_ylim([ - 15, res_hight + 2])
    
    # get polygon up
    polygon_up = get_polygon(x_p_vec[i], 'up', c_up=up_colors[i])

    # get polygon down
    # print(down_colors[i])
    polygon_down = get_polygon(x_p_vec[i], 'down', c_down=down_colors[i])

    # get piston polygon
    polygon_piston = get_polygon(x_p_vec[i], 'piston')

    # get static polygons
    polygons_res = get_polygon(x_p_vec[i], 'reservars')

    # get background polygon
    polygon_back = get_polygon(x_p_vec[i], 'background')

    # get vent polygon
    polygon_vent = get_polygon(x_p_vec[i], 'vent', x_th=x_th_vec[i])

    # get nozle polygon 
    polygons_nozle = get_polygon(x_p_vec[i], 'nozle', p_down=p_down_vec[i], c_down=down_colors[i])

    drop_flag = False

    if len(drops_time) > 0:
        drops_time = [t+1 for t in drops_time]

    #jet drowing
    if l_jet_vec[i] > l_critic:
        
        l_jet = l_jet_vec[i]
        i_drop_c = 1
        while l_jet > l_critic:

            l_jet = l_jet - l_critic
            if i_drop_c > len(drops_time):
                # print(l_jet_vec[i])
                drops_time.append(0)
                drops_init_vel.append(v_jet_vec[i])
            i_drop_c += 1            
    else:
        l_jet = l_jet_vec[i]


    # get jet polygon 
    if l_jet > 0:
        polygon_jet = get_polygon(x_p_vec[i], 'jet', l_jet=l_jet)

    # drop drowing
    # get time of droping
    if len(drops_time) > 0:
        # get velocity 
        # global circles_drops
        circles_drops = get_circles()
        
        
    axes[1].add_patch(polygon_back)

    for stat_polygon in polygons_res:
        axes[1].add_patch(stat_polygon)

    for nozle_polygon in polygons_nozle:
        axes[1].add_patch(nozle_polygon)

    if l_jet > 0:
        axes[1].add_patch(polygon_jet)

    if len(drops_time) > 0:
        # print(drops_time)
        # print(len(circles_drops))
        for crcl_drop in circles_drops:
            axes[1].add_patch(crcl_drop)

    axes[1].add_patch(polygon_up)
    axes[1].add_patch(polygon_down)
    axes[1].add_patch(polygon_piston)
    axes[1].add_patch(polygon_vent)

    return axes

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=time.shape[0], interval=40) #, blit=True)

# To save the animation using Pillow as a gif
# writer = animation.PillowWriter(fps=10,
#                                 metadata=dict(artist='Me'),
#                                 bitrate=1800)

# anim.save('pistons.gif', writer=writer)

# saving to m4 using ffmpeg writer 
# writervideo = animation.FFMpegWriter(fps=60) 
# anim.save('pistons.mp4', writer=writervideo) 

plt.show()

/tmp/ipykernel_54737/1896701285.py:312: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  axes[j].set_ylim([plot_dict[j]['y_min'], plot_dict[j]['y_max']])


In [14]:
import matplotlib as mpl

cmap = mpl.colormaps['plasma']
colors = cmap(p_down_vec/p_down_vec.max()) # convert nfloors to colors that we can use later
print(p_down_vec)
print(colors)
print(up_colors)


[81.         73.59833795 66.55124654 59.85872576 53.52077562 47.53739612
 41.90858726 36.63434903 31.71468144 27.14958449 22.93905817 19.08310249
 15.58171745 12.43490305  9.64265928  7.20498615  5.12188366  3.3933518
  2.01939058  1.        ]
[[9.40015e-01 9.75158e-01 1.31326e-01 1.00000e+00]
 [9.86509e-01 8.22401e-01 1.43557e-01 1.00000e+00]
 [9.92541e-01 6.87030e-01 1.92170e-01 1.00000e+00]
 [9.68526e-01 5.69700e-01 2.61721e-01 1.00000e+00]
 [9.25825e-01 4.68103e-01 3.29435e-01 1.00000e+00]
 [8.72303e-01 3.78774e-01 3.93355e-01 1.00000e+00]
 [8.12612e-01 2.97928e-01 4.55338e-01 1.00000e+00]
 [7.48289e-01 2.22711e-01 5.16834e-01 1.00000e+00]
 [6.83758e-01 1.56278e-01 5.71660e-01 1.00000e+00]
 [6.10667e-01 9.02040e-02 6.19951e-01 1.00000e+00]
 [5.40570e-01 3.49500e-02 6.48640e-01 1.00000e+00]
 [4.71457e-01 5.67800e-03 6.59897e-01 1.00000e+00]
 [4.05503e-01 6.78000e-04 6.56977e-01 1.00000e+00]
 [3.43925e-01 4.99100e-03 6.44710e-01 1.00000e+00]
 [2.87076e-01 1.08550e-02 6.27295e-01 1.00

In [8]:

import matplotlib as mpl

n_lines = 21
cmap = mpl.colormaps['plasma']

# Take colors at regular intervals spanning the colormap.
colors = cmap(np.linspace(0, 1, n_lines))
print(colors)

[[5.03830e-02 2.98030e-02 5.27975e-01 1.00000e+00]
 [1.64070e-01 2.01710e-02 5.77478e-01 1.00000e+00]
 [2.54627e-01 1.38820e-02 6.15419e-01 1.00000e+00]
 [3.37683e-01 5.61800e-03 6.43049e-01 1.00000e+00]
 [4.17642e-01 5.64000e-04 6.58390e-01 1.00000e+00]
 [4.94877e-01 1.19900e-02 6.57865e-01 1.00000e+00]
 [5.62738e-01 5.15450e-02 6.41509e-01 1.00000e+00]
 [6.31017e-01 1.07699e-01 6.08287e-01 1.00000e+00]
 [6.92840e-01 1.65141e-01 5.64522e-01 1.00000e+00]
 [7.48289e-01 2.22711e-01 5.16834e-01 1.00000e+00]
 [7.98216e-01 2.80197e-01 4.69538e-01 1.00000e+00]
 [8.40155e-01 3.33580e-01 4.27455e-01 1.00000e+00]
 [8.81443e-01 3.92529e-01 3.83229e-01 1.00000e+00]
 [9.18109e-01 4.53603e-01 3.39529e-01 1.00000e+00]
 [9.49217e-01 5.17763e-01 2.95662e-01 1.00000e+00]
 [9.73416e-01 5.85761e-01 2.51540e-01 1.00000e+00]
 [9.88260e-01 6.52325e-01 2.11364e-01 1.00000e+00]
 [9.94553e-01 7.28728e-01 1.71622e-01 1.00000e+00]
 [9.88648e-01 8.09579e-01 1.45357e-01 1.00000e+00]
 [9.68443e-01 8.94564e-01 1.470

In [28]:
nozzle_coords

[array([[-0.4       , -1.        ],
        [-0.4       , -1.88888889],
        [ 0.4       , -1.88888889],
        [ 0.4       , -1.        ]]),
 array([[-0.4       , -0.88888889],
        [-0.4       , -1.66666667],
        [ 0.4       , -1.66666667],
        [ 0.4       , -0.88888889]]),
 array([[-0.4       , -0.77777778],
        [-0.4       , -1.44444444],
        [ 0.4       , -1.44444444],
        [ 0.4       , -0.77777778]]),
 array([[-0.4       , -0.66666667],
        [-0.4       , -1.22222222],
        [ 0.4       , -1.22222222],
        [ 0.4       , -0.66666667]]),
 array([[-0.4       , -0.55555556],
        [-0.4       , -1.        ],
        [ 0.4       , -1.        ],
        [ 0.4       , -0.55555556]]),
 array([[-0.4       , -0.44444444],
        [-0.4       , -0.77777778],
        [ 0.4       , -0.77777778],
        [ 0.4       , -0.44444444]]),
 array([[-0.4       , -0.33333333],
        [-0.4       , -0.55555556],
        [ 0.4       , -0.55555556],
        [ 0.4   